# Project to extract, collect and save tweets using Twitter API's into MongoDB

## Installing Python libraries required 

We may need to restart our Jupyter Notebook instance from AWS after installed those libraries. **Please make sure AWS instance is stopped after extraction of tweets.**

In [3]:
!pip install pymongo

     |████████████████████████████████| 450 kB 15.1 MB/s            


In [4]:
!pip install pymongo[srv]

     |████████████████████████████████| 269 kB 21.6 MB/s            


In [5]:
!pip install dnspython

In [6]:
!pip install tweepy

     |████████████████████████████████| 69 kB 8.3 MB/s             
  Using cached requests-2.27.1-py2.py3-none-any.whl (63 kB)
     |████████████████████████████████| 151 kB 33.4 MB/s            
  Attempting uninstall: requests
    Found existing installation: requests 2.26.0
    Uninstalling requests-2.26.0:
      Successfully uninstalled requests-2.26.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
anaconda-project 0.9.1 requires ruamel-yaml, which is not installed.


In [8]:
!pip install twitter

## Import Python libraries into the notebook instance
reference : https://medium.com/i%CC%87stanbuldatascienceacademy/fetching-data-from-twitter-to-mongodb-9cbf55a24361

In [2]:
import pymongo
from pymongo import MongoClient
import json
import tweepy
import twitter
from pprint import pprint
import configparser
import pandas as pd

##  Load the Authorization Information required for Twitter and MongoDB

Save twitter authorization and database connection information and API Keys in a config.ini file.
Use the configparser to load the authorization and MongoDB cloud link information. 

In [3]:
# Use configparser to get details from config.ini file
config = configparser.ConfigParser()
config.read('config.ini')

#Details to use Twitter API's
consumer_key      = config['twitterdetails']['api_key']        #"B8THVc5cG9XT352xbaDYG1Zbn"
consumer_secret   = config['twitterdetails']['api_secrete']    #"xEF30YWlmEE2neqJh8JM0Tfpio9UGhbSuO4jpMymnC7CJDlVwk" 
oauth_token       = config['twitterdetails']['access_token']   #"2337780624-cmH5WjCtzp9sUhaXE2Vl1xkwPITUT0FMWEARHvp" 
oauth_token_secret = config['twitterdetails']['access_secrete'] #"md2MwgY6affvJEbeMqQsMsWrAzIoDyocyLg5cicqjtFKn" 

#Details to connect to MongoDB Cloud Cluster
mongod_connect = config['mongodetails']['connection']

## Connect to the MongoDB Cloud Cluster , create a database, collection and an index in MongoDB cloud using the below code.
Cluster to be created in MongoDB intance

In [4]:
# Get MongoDB Cluster connection
client = MongoClient(mongod_connect)
# Use or create a database named demo
db = client.demo 
# Create a collection tweet_collection
tweet_collection = db.tweet_collection
# Create an index with "id" and make sure the collected tweets are unique
tweet_collection.create_index([("id", pymongo.ASCENDING)],unique = True) 

## Using the REST API to Collect static Tweets

Getting the rest_auth instance and using it for fetching REST API instance

In [4]:
# Get an REST Authorization instance using the Twitter Developer Project Details
rest_auth = twitter.oauth.OAuth(oauth_token,oauth_token_secret,consumer_key,consumer_secret)
rest_api = twitter.Twitter(auth=rest_auth)

Get all the tweets based on the geolocation , count and search keyword
Query with the help of REST API. Provide Geolocation of Windsor from Google Maps , provide maximum range of 50 Kilometers and with "covid" keyword.

In [6]:
# Number of tweets that needs to be returned from REST API. Default and Maximum value is 100.
count = 100 
# Latitude value of Windsor , ON
latitude = 42.2949797      
# Longitude value of Windsor, ON
longitude = -83.0729219    
# Search Range of tweets in Kilometers
max_range = 50             
#geocode = "42.2949797, -83.0729219, 50km"  #  location of Windsor, ON , Retrieved with the help of Google Maps
geocode = "%f,%f,%dkm" % (latitude, longitude, max_range)
# Search Keyword - COVID
q = "covid"                               
# Get the Search Results using the above values
search_results = rest_api.search.tweets( count = count, q = q, geocode = geocode) 
# Get all the tweets from the JSON response
statuses = search_results["statuses"]

# Below Print required during debug 
#pprint(search_results["statuses"])
#pprint(tweet_collection)
#pprint(statuses[1])

Saving the results from the twitter API into MongoDB Cloud instance
Loop all the results and insert the records into the MongoDB cloud instance

In [7]:
# Below ID to be used to extract further data and saving the tweets into MongoDB
since_id_new = statuses[-1]['id']
# Loop all the result 
for statuse in statuses:
    try:
        #print(statuse) # Used during debug to check the values
        tweet_collection.insert_one(statuse) # Inserting an indivual value into MongoDB instance
        pprint(statuse['created_at'])# print the date of the collected tweets , so that we know how many tweets have been inserted into MongoDB
    except:
        pass

'Fri Apr 01 21:15:58 +0000 2022'
'Fri Apr 01 21:10:32 +0000 2022'
'Fri Apr 01 21:09:59 +0000 2022'
'Fri Apr 01 21:08:08 +0000 2022'
'Fri Apr 01 21:07:05 +0000 2022'
'Fri Apr 01 21:01:14 +0000 2022'
'Fri Apr 01 21:00:29 +0000 2022'
'Fri Apr 01 20:50:01 +0000 2022'
'Fri Apr 01 20:49:58 +0000 2022'
'Fri Apr 01 20:47:54 +0000 2022'
'Fri Apr 01 20:36:57 +0000 2022'
'Fri Apr 01 20:29:20 +0000 2022'
'Fri Apr 01 20:24:00 +0000 2022'
'Fri Apr 01 20:22:54 +0000 2022'
'Fri Apr 01 20:14:15 +0000 2022'
'Fri Apr 01 20:11:07 +0000 2022'
'Fri Apr 01 20:07:35 +0000 2022'
'Fri Apr 01 20:00:48 +0000 2022'
'Fri Apr 01 20:00:08 +0000 2022'
'Fri Apr 01 19:59:06 +0000 2022'
'Fri Apr 01 19:43:41 +0000 2022'
'Fri Apr 01 19:40:01 +0000 2022'
'Fri Apr 01 19:35:33 +0000 2022'
'Fri Apr 01 19:35:29 +0000 2022'
'Fri Apr 01 19:35:02 +0000 2022'
'Fri Apr 01 19:31:15 +0000 2022'
'Fri Apr 01 19:30:03 +0000 2022'
'Fri Apr 01 19:27:37 +0000 2022'
'Fri Apr 01 19:25:35 +0000 2022'
'Fri Apr 01 19:24:44 +0000 2022'
'Fri Apr 0

Below code continues fetching tweets with the same query. We need to stop executing manually after some time , since our MongoDB free cloud instance limit would be full very quickly. 

In [ ]:
# Fetch Further queries and stop the execution after some time , once you feel you have enough tweets to perform visualization
since_id_old = 0
while(since_id_new != since_id_old):
    since_id_old = since_id_new
    search_results = rest_api.search.tweets( count=count,q=q,
                        geocode=geocode, max_id= since_id_new)
    statuses = search_results["statuses"]
    since_id_new = statuses[-1]['id']
    for statuse in statuses:
        try:
            tweet_collection.insert_one(statuse)
            pprint(statuse['created_at']) # print the date of the collected tweets
        except:
            pass

'Fri Apr 01 15:37:00 +0000 2022'
'Fri Apr 01 15:36:44 +0000 2022'
'Fri Apr 01 15:27:13 +0000 2022'
'Fri Apr 01 15:19:10 +0000 2022'
'Fri Apr 01 15:07:04 +0000 2022'
'Fri Apr 01 15:04:12 +0000 2022'
'Fri Apr 01 15:03:22 +0000 2022'
'Fri Apr 01 15:02:33 +0000 2022'
'Fri Apr 01 15:01:00 +0000 2022'
'Fri Apr 01 15:00:08 +0000 2022'
'Fri Apr 01 14:58:16 +0000 2022'
'Fri Apr 01 14:56:07 +0000 2022'
'Fri Apr 01 14:49:27 +0000 2022'
'Fri Apr 01 14:36:49 +0000 2022'
'Fri Apr 01 14:36:30 +0000 2022'
'Fri Apr 01 14:34:40 +0000 2022'
'Fri Apr 01 14:34:38 +0000 2022'
'Fri Apr 01 14:33:41 +0000 2022'
'Fri Apr 01 14:17:07 +0000 2022'
'Fri Apr 01 14:13:35 +0000 2022'
'Fri Apr 01 14:06:46 +0000 2022'
'Fri Apr 01 14:02:09 +0000 2022'
'Fri Apr 01 14:00:24 +0000 2022'
'Fri Apr 01 14:00:21 +0000 2022'
'Fri Apr 01 14:00:04 +0000 2022'
'Fri Apr 01 13:59:24 +0000 2022'
'Fri Apr 01 13:55:26 +0000 2022'
'Fri Apr 01 13:40:27 +0000 2022'
'Fri Apr 01 13:33:49 +0000 2022'
'Fri Apr 01 13:28:09 +0000 2022'
'Fri Apr 0

'Thu Mar 31 14:37:15 +0000 2022'
'Thu Mar 31 14:35:39 +0000 2022'
'Thu Mar 31 14:34:41 +0000 2022'
'Thu Mar 31 14:32:14 +0000 2022'
'Thu Mar 31 14:31:09 +0000 2022'
'Thu Mar 31 14:30:02 +0000 2022'
'Thu Mar 31 14:28:54 +0000 2022'
'Thu Mar 31 14:28:37 +0000 2022'
'Thu Mar 31 14:15:00 +0000 2022'
'Thu Mar 31 14:12:45 +0000 2022'
'Thu Mar 31 14:12:03 +0000 2022'
'Thu Mar 31 14:10:28 +0000 2022'
'Thu Mar 31 14:00:13 +0000 2022'
'Thu Mar 31 13:59:53 +0000 2022'
'Thu Mar 31 13:50:44 +0000 2022'
'Thu Mar 31 13:47:25 +0000 2022'
'Thu Mar 31 13:45:00 +0000 2022'
'Thu Mar 31 13:42:51 +0000 2022'
'Thu Mar 31 13:15:28 +0000 2022'
'Thu Mar 31 13:04:59 +0000 2022'
'Thu Mar 31 13:01:57 +0000 2022'
'Thu Mar 31 12:57:04 +0000 2022'
'Thu Mar 31 12:44:58 +0000 2022'
'Thu Mar 31 12:26:07 +0000 2022'
'Thu Mar 31 12:24:58 +0000 2022'
'Thu Mar 31 12:22:26 +0000 2022'
'Thu Mar 31 12:18:07 +0000 2022'
'Thu Mar 31 12:11:33 +0000 2022'
'Thu Mar 31 12:09:51 +0000 2022'
'Thu Mar 31 11:51:42 +0000 2022'
'Thu Mar 3

'Wed Mar 30 11:04:06 +0000 2022'
'Wed Mar 30 11:00:39 +0000 2022'
'Wed Mar 30 11:00:11 +0000 2022'
'Wed Mar 30 10:36:03 +0000 2022'
'Wed Mar 30 10:31:42 +0000 2022'
'Wed Mar 30 10:05:42 +0000 2022'
'Wed Mar 30 09:47:53 +0000 2022'
'Wed Mar 30 08:08:09 +0000 2022'
'Wed Mar 30 07:20:40 +0000 2022'
'Wed Mar 30 07:16:05 +0000 2022'
'Wed Mar 30 06:47:41 +0000 2022'
'Wed Mar 30 05:57:32 +0000 2022'
'Wed Mar 30 05:31:05 +0000 2022'
'Wed Mar 30 05:28:31 +0000 2022'
'Wed Mar 30 04:32:14 +0000 2022'
'Wed Mar 30 04:25:41 +0000 2022'
'Wed Mar 30 04:18:06 +0000 2022'
'Wed Mar 30 04:09:57 +0000 2022'
'Wed Mar 30 04:06:01 +0000 2022'
'Wed Mar 30 04:05:15 +0000 2022'
'Wed Mar 30 03:41:21 +0000 2022'
'Wed Mar 30 03:35:00 +0000 2022'
'Wed Mar 30 03:30:46 +0000 2022'
'Wed Mar 30 03:26:23 +0000 2022'
'Wed Mar 30 03:14:13 +0000 2022'
'Wed Mar 30 03:10:51 +0000 2022'
'Wed Mar 30 03:10:00 +0000 2022'
'Wed Mar 30 02:42:14 +0000 2022'
'Wed Mar 30 02:33:48 +0000 2022'
'Wed Mar 30 02:31:48 +0000 2022'
'Wed Mar 3

'Tue Mar 29 07:13:04 +0000 2022'
'Tue Mar 29 06:45:03 +0000 2022'
'Tue Mar 29 05:20:48 +0000 2022'
'Tue Mar 29 04:30:45 +0000 2022'
'Tue Mar 29 04:30:18 +0000 2022'
'Tue Mar 29 04:12:28 +0000 2022'
'Tue Mar 29 03:47:34 +0000 2022'
'Tue Mar 29 03:44:20 +0000 2022'
'Tue Mar 29 03:43:30 +0000 2022'
'Tue Mar 29 03:41:32 +0000 2022'
'Tue Mar 29 03:26:04 +0000 2022'
'Tue Mar 29 03:22:40 +0000 2022'
'Tue Mar 29 03:15:27 +0000 2022'
'Tue Mar 29 03:15:27 +0000 2022'
'Tue Mar 29 03:06:02 +0000 2022'
'Tue Mar 29 02:54:03 +0000 2022'
'Tue Mar 29 02:54:02 +0000 2022'
'Tue Mar 29 02:50:22 +0000 2022'
'Tue Mar 29 02:42:23 +0000 2022'
'Tue Mar 29 02:34:05 +0000 2022'
'Tue Mar 29 02:17:21 +0000 2022'
'Tue Mar 29 02:02:57 +0000 2022'
'Tue Mar 29 01:48:26 +0000 2022'
'Tue Mar 29 01:42:44 +0000 2022'
'Tue Mar 29 01:37:56 +0000 2022'
'Tue Mar 29 01:24:15 +0000 2022'
'Tue Mar 29 01:09:52 +0000 2022'
'Tue Mar 29 00:51:17 +0000 2022'
'Tue Mar 29 00:37:38 +0000 2022'
'Tue Mar 29 00:20:40 +0000 2022'
'Tue Mar 2

'Mon Mar 28 00:57:00 +0000 2022'
'Mon Mar 28 00:55:31 +0000 2022'
'Mon Mar 28 00:53:22 +0000 2022'
'Mon Mar 28 00:53:22 +0000 2022'
'Mon Mar 28 00:53:20 +0000 2022'
'Mon Mar 28 00:53:17 +0000 2022'
'Mon Mar 28 00:51:50 +0000 2022'
'Mon Mar 28 00:49:37 +0000 2022'
'Mon Mar 28 00:46:22 +0000 2022'
'Mon Mar 28 00:45:33 +0000 2022'
'Mon Mar 28 00:44:48 +0000 2022'
'Mon Mar 28 00:42:09 +0000 2022'
'Mon Mar 28 00:42:08 +0000 2022'
'Mon Mar 28 00:42:05 +0000 2022'
'Mon Mar 28 00:38:25 +0000 2022'
'Mon Mar 28 00:33:37 +0000 2022'
'Mon Mar 28 00:32:19 +0000 2022'
'Mon Mar 28 00:28:42 +0000 2022'
'Mon Mar 28 00:22:52 +0000 2022'
'Mon Mar 28 00:22:19 +0000 2022'
'Mon Mar 28 00:16:40 +0000 2022'
'Mon Mar 28 00:14:57 +0000 2022'
'Mon Mar 28 00:13:35 +0000 2022'
'Mon Mar 28 00:06:27 +0000 2022'
'Mon Mar 28 00:02:33 +0000 2022'
'Sun Mar 27 23:57:46 +0000 2022'
'Sun Mar 27 23:54:14 +0000 2022'
'Sun Mar 27 23:36:10 +0000 2022'
'Sun Mar 27 23:28:59 +0000 2022'
'Sun Mar 27 23:12:22 +0000 2022'
'Sun Mar 2

'Sat Mar 26 14:54:02 +0000 2022'
'Sat Mar 26 14:52:53 +0000 2022'
'Sat Mar 26 14:52:53 +0000 2022'
'Sat Mar 26 14:52:03 +0000 2022'
'Sat Mar 26 14:49:04 +0000 2022'
'Sat Mar 26 14:41:41 +0000 2022'
'Sat Mar 26 14:37:09 +0000 2022'
'Sat Mar 26 14:25:27 +0000 2022'
'Sat Mar 26 14:20:58 +0000 2022'
'Sat Mar 26 14:20:54 +0000 2022'
'Sat Mar 26 14:20:26 +0000 2022'
'Sat Mar 26 14:18:39 +0000 2022'
'Sat Mar 26 14:07:43 +0000 2022'
'Sat Mar 26 14:05:48 +0000 2022'
'Sat Mar 26 13:54:59 +0000 2022'
'Sat Mar 26 13:49:08 +0000 2022'
'Sat Mar 26 13:47:23 +0000 2022'


## Display all the Collected Tweets for Verification

In [5]:
# Display the number of tweets collected
print(tweet_collection.estimated_document_count())

# Display the unique twitter users for the saved tweets , just to get an idea about the data
user_cursor = tweet_collection.distinct("user.id")
print (len(user_cursor)) 

1567
842


In [6]:
# Create a text index 
tweet_collection.create_index([("text", pymongo.TEXT)], name='text_index', default_language='english') 


'text_index'

In [8]:
# Create a cursor to query tweets with the text index that has been created in the above statement
tweet_cursor = tweet_collection.find({"$text": {"$search": "covid"}}) # return tweets contain covid

Using the cursor , display texts of the tweets for the verification

In [9]:

# Display only first 10 tweets using the cursor created above
for document in tweet_cursor[0:10]: 
    try:
        print ('----')
#         pprint (document) # use pprint to print the entire tweet document
        print ('name:', document["user"]["name"]) # user name
        print ('text:', document["text"])         # tweets
    except:
        print ("***error in encoding")
        pass

----
name: Joy Alekman (she/her/bitch)
text: Covid!
----
name:  KashJuan
text: I have not caught covid yet cause of mushrooms 🍄 😂🖕🏾 fuck you covid-19!
----
name: Marc Tucker
text: @1NAMILLCARAMEL On my way home we had to take Covid-19test because I was sick just a head cold no Covid
----
name: jenifer is on a #digitaldiet
text: Long COVID symptoms might depend on the COVID variant https://t.co/iJCCgosS3a
----
name: Belle Isle Trash
text: I took my first at-home Joe Brandon COVID test and it turns out I'm 100% that bitch and COVID free.
----
name: Behind the Scenes 🎥
text: Fully Vaccinated and caught COVID again ! I can honestly say second time around was easier for me ! #covid #FridayFeeling
----
name: Lori Jo Vest 💕
text: Y'all, you might be done with COVID but COVID isn't done with you. I'm staying home, staying masked when I'm out an… https://t.co/ezap8DvHE0
----
name: Jaymac
text: @ynakhooda @StevenDelDuca Well the pic says it all Covid is over and the fear  and hype with it.   Co